# Задача 1. Использовать паттерн проектирования "Наблюдатель"

In [1]:
from abc import ABC, abstractmethod
from typing import List, Tuple
import random


class PetFoodWarehouse:

    """
    Класс хранения еды на складе.
    all_items = {} словарь для хранения всех экземплаяров класса (т.е складские позиции)
    self.name = name - название поступившего корма
    self.quantity = quantity - кол-во корма, кг
    """
    all_items = {}  

    def __init__(self, name, quantity):
        self.name = name
        self.quantity = quantity
        self.__class__.all_items[self.name] = self  

    def __str__(self):
        return f"{self.name}: {self.quantity} кг"


class PetFood(ABC):
    """
    Класс-интерфейс издателя объявляет методы для управления подписчиками.Абстрактен
    Методы:
           attach(self, observer: "Observer")  - подключение подписчика
           detach(self, observer: "Observer") - отключение подписчика
           notify(self) - уведомление подписчиков
    """
    
    @abstractmethod
    def attach(self, observer: "Observer") -> None:
        pass

    @abstractmethod
    def detach(self, observer: "Observer") -> None:
        pass

    @abstractmethod
    def notify(self) -> None:
        pass


class ArrivalPetFood(PetFood):

    """
    Класс-издаетель выполняет посутпление еды на склад и вызов оповещние при поступлении 
    конкретного типа корма
    Параметры: 
        _observers: List["Observer"] = []   Общий список подписчиков для всех экземпляров
        self.list_food = list_food  Список поставки [(название, количество)]
        self._type_food: str = None   Текущий тип поступившего корма
        self._observers: List[Observer] = []  Список подписчиков
    Методы:
           attach(self, observer: "Observer")  - подключение подписчика
           detach(self, observer: "Observer") - отключение подписчика
           notify(self) - Уведомляет подписчиков о поступлении нового корма.
           show_observers(self) -  Метод отображения текущих подписчиков
           arriaval_to_warehouse - метод обработки поставки:
                                        1. Добавляет товар на склад (создает экземпляры PetFoodWarehouse).
                                        2. Уведомляет подписчиков о наличии корма по типу.
    """
    _observers: List["Observer"] = [] 
    
    def __init__(self, list_food: List[Tuple[str, int]]):
        self.list_food = list_food  
        self._type_food: str = None  


    def attach(self, observer: "Observer") -> None:
        print(f"Подписчик подключен на уведомления.")
        self.__class__._observers.append(observer)

    def detach(self, observer: "Observer") -> None:
        print(f"Подписчик отключен от уведомлений.")
        self.__class__._observers.remove(observer)

    def notify(self) -> None:
        print("Оповещение подписчиков...")
        for observer in self.__class__._observers:
            observer.update(self)

    def show_observers(self):
        print("Текущие подписчики:")
        for obs in self.__class__._observers:
            print(obs)

    def arriaval_to_warehouse(self) -> None:

        for food_name, quantity in self.list_food:
            print(f"Добавляем на склад: {food_name}, {quantity} кг")
            PetFoodWarehouse(food_name, quantity)

            self._type_food = food_name
            self.notify()


class Observer(ABC):
    """
    Класс-интерфейс Наблюдателя объявляет метод уведомления.
    """
    @abstractmethod
    def update(self, petfood: PetFood) -> None:
        pass


class FarminaObserver(Observer):
    """
    Подкласс наблюдателя уведомляет о поступлении корма Farmina.
    """
    def update(self, petfood: PetFood) -> None:
        if petfood._type_food == "Корм Farmina":
            print("Поступил корм Farmina!")


class VetPetObserver(Observer):
    """
    Подкласс наблюдателя уведомляет о поступлении корма VetPet.
    """
    def update(self, petfood: PetFood) -> None:
        if petfood._type_food == "Корм VetPet":
            print("Поступил корм VetPet!")



    

In [2]:
# Создаем подписчиков
observer_a = FarminaObserver()
observer_b = VetPetObserver()
observer_c = VetPetObserver()


# Создаем издателя с поставкой
supply = ArrivalPetFood([("Корм Farmina", 10), ("Корм VetPet", 20)])
supply.show_observers()

# Подключаем подписчиков
supply.attach(observer_a)
supply.attach(observer_b)
supply.attach(observer_c)

supply.show_observers()
# Выполняем поступление груза на склад
supply.arriaval_to_warehouse()

# Выводим содержимое склада
print("\nТекущее содержимое склада:")
for item in PetFoodWarehouse.all_items.values():
    print(item)
supply.show_observers()
print("\n")
supply.detach(observer_c)

supply2 = ArrivalPetFood([("Корм VetPet", 20)])
supply.show_observers()
# Выполняем поступление груза на склад
supply2.arriaval_to_warehouse()


Текущие подписчики:
Подписчик подключен на уведомления.
Подписчик подключен на уведомления.
Подписчик подключен на уведомления.
Текущие подписчики:
Добавляем на склад: Корм Farmina, 10 кг
Оповещение подписчиков...
Поступил корм Farmina!
Добавляем на склад: Корм VetPet, 20 кг
Оповещение подписчиков...
Поступил корм VetPet!
Поступил корм VetPet!

Текущее содержимое склада:
Корм Farmina: 10 кг
Корм VetPet: 20 кг
Текущие подписчики:


Подписчик отключен от уведомлений.
Текущие подписчики:
Добавляем на склад: Корм VetPet, 20 кг
Оповещение подписчиков...
Поступил корм VetPet!
